# Imports

In [1]:
using Combinatorics
include("../src/SW.jl");

  Activating project at `~/Projects/SW`


In [2]:
function find_inds(nInit, order::Int, V1)
    is = nInit
    curr_inds = nInit

    for o in 1:order
        next_inds = []
        for i in curr_inds
            inds, vals = findnz(V1[i,:])
            push!(next_inds, inds)
        end

        curr_inds = filter(x->!(x ∈ is), rmDoubles(vcat(next_inds...)))
        append!(is, curr_inds)
    end

    return rmDoubles(vcat(is...))
end

find_inds (generic function with 1 method)

# 1st try: generate new configs from known configurations

Doesn't work since that way all the basis states are generated

In [5]:
function iterate_through_diff(diff)
    pos = []
    for (n,s) in enumerate(diff)
        abs(s) > 0 && push!(pos, n)
    end
    return Combinatorics.combinations(pos)
end;

function generate_configs(spin, pos)
    return map(pos) do p
        return spinflip(spin, p)
    end
end

generate_configs (generic function with 1 method)

In [ ]:
iter = iterate_through_diff(abs.(spin_basis[476] .- spin_basis[205]))
generate_configs(spin_basis[476], iter)

In [ ]:
# is = [i for (i,dw) in enumerate(dw_precalc) if dw == 8];
is = [145,205,251,285,336,370,416,476]

configs = Dict()
for (i,j) in Iterators.product(is,is)
    j >= i && continue
    for c in generate_configs(spin_basis[i], iterate_through_diff(abs.(spin_basis[i] .- spin_basis[j])))
        configs[c] = true
        # c ∈ configs && continue
        # push!(configs, c)
    end
    
    # append!(configs, generate_configs(spin_basis[i], iterate_through_diff(abs.(spin_basis[i] .- spin_basis[j]))))
end
configs

# 2nd try: Take only N=8,10,12 spin-basis states and diagonalize resulting hamiltonian

Doesn't really work

In [3]:
L= (4,4)
N = prod(L);
J = - 1;
g = - 0.5
h = - 0.

next_neighbours = nearest_neighbours(L, collect(1:prod(L)))

spin_basis = vec(collect(Iterators.product(fill([1,0],N)...)));
dw_precalc = map(spin_basis) do spin
    return domainWallL(spin, L)
end

spin_basis_tableFull = Dict(
    map(enumerate(zip(spin_basis,dw_precalc))) do (i, (spin, dw))
        return (spin, (i,dw))
    end
);


### sort basis according to domain wall length ###
sorted_spin_basis = sort(collect(zip(dw_precalc, spin_basis)), by = x->x[1])
dw_precalc  = [d[1] for d in sorted_spin_basis]
spin_basis  = [d[2] for d in sorted_spin_basis];
ts = [0.0,0.1]
step = 1.1
tmax = 1e10

### logarithmic timesteps ###
while true
    push!(ts, ts[end]*step)
    ts[end] > tmax && break
end

# is6 = [i for (i,dw) in enumerate(dw_precalc) if dw == 6];
# is8 = [i for (i,dw) in enumerate(dw_precalc) if dw == 8];
# is10 = [i for (i,dw) in enumerate(dw_precalc) if dw == 10];
# is12 = [i for (i,dw) in enumerate(dw_precalc) if dw == 12];
# is14 = [i for (i,dw) in enumerate(dw_precalc) if dw == 14];
# is = vcat(is6,is8,is10,is12)
# @show length(is6)
# @show length(is8)
# @show length(is10)
# @show length(is12)
# @show length(is14)

HInds, RInds = build_H1_R1(spin_basis, spin_basis_tableFull, (L,J,g,h));
VInds = HInds+RInds;
init_states_indices = [145,205,251,285,336,370,416,476]
is = find_inds(init_states_indices, 4, VInds)
@show length(is)

dw_precalc  = [dw_precalc[i] for i in is]
spin_basis  = [spin_basis[i] for i in is];
# @show dw_precalc

spin_basis_table = Dict(
    map(enumerate(zip(spin_basis,dw_precalc))) do (i, (spin, dw))
        return (spin, (i,dw))
    end
);

domainWall_table = Dict(
    map(enumerate(dw_precalc)) do (i,  dw)
        return (i, dw)
    end
);

### initial state ###
init_spin = vcat(fill(1,Int(N/2)),fill(0,Int(N/2)))
init_idx = first(spin_basis_table[Tuple(init_spin)])

psi=zeros(length(spin_basis))
psi[init_idx]=1
psi = sparse(psi)

### build Hamiltonians ###
H0  = build_H0(spin_basis, next_neighbours, spin_basis_table, (L,J,g,h));
H1, R1 = build_H1_R1(spin_basis, spin_basis_table, (L,J,g,h));
V1 = H1+R1;
H = H0+V1

println("Starting caluclation of eigenvectors/-values")
@time "eigen: " vals, vecs = eigen(Matrix(H))
dw = map(1:size(vecs)[2]) do i
    absVals = abs2.(vecs[:,i])
    return sum(absVals .* dw_precalc)
end

dfSpec = DataFrame(en = real.(vals), dw = dw, occ = real.(vecs[init_idx,:]))
CSV.write("./spec_Eff_L=($(L[1])_$(L[2]))_J=$(J)_g=$(g)_h=$(h).csv", dfSpec)

psi = Transpose(vecs) * psi

imb_precalc = map(s -> imbalance(s, L), spin_basis)
data = Any[]

for (t, tf) in zip(ts[1:end-1], ts[2:end])
    dt = tf-t

    psi_prime = vecs*psi

    imb = mapreduce(+, enumerate(psi_prime)) do (i,psi_i)
        α = abs2(psi_i)
        return α * imb_precalc[i]
    end

    dwObs = real.(dot(psi_prime, H0 * psi_prime))

    append!(data, [[t, imb, dwObs]])

    # Propagate state
    U = exp.(-1im*dt .* vals)
    psi = U .* psi
#vecs = sort([(i,p) for (i,p) in enumerate(abs2.(psi_prime)) if p>1e-3], by=x->x[2], rev=true)
#@show [v[1] for v in vecs]
end

df = DataFrame(t = [real(d[1]) for d in data], imb = [real(d[2]) for d in data], N = [real(d[3]) for d in data])
CSV.write("./obs_Eff_new_L=($(L[1])_$(L[2]))_J=$(J)_g=$(g)_h=$(h).csv", df)

length(is) = 12817
Starting caluclation of eigenvectors/-values


eigen: 

: 2209.115584 seconds (1.23 M allocations: 4.980 GiB, 0.01% gc time, 0.02% compilation time)


"./obs_Eff_new_L=(4_4)_J=-1_g=-0.5_h=-0.0.csv"

In [8]:
findmax(abs.(dfSpec[:,"occ"]))

(0.9068857214208457, 197)

In [9]:
dfSpec[findmax(abs.(dfSpec[:,"occ"]))[2],:]

Row,en,dw,occ
,Float64,Float64,Float64
197,-16.7318,8.34399,-0.906886


Compare spectra of effective and full model

### Generalize building spin states with fixed domain wall length for any lattice size

In an $L \times L$ lattice with PBC, the number of possibilities to place N domain walls is given by $\frac{(2L^2) !}{N! (2L^2-N)!}$. However, out of those we need to pick out those that are connected.

In [15]:
function nPossibleStates(L::Int, N::Int)
    return binomial(2*L^2, N)
end

nPossibleStates (generic function with 1 method)

In [18]:
nPossibleStates(8,8)

1429702652400

In [19]:
spin_basis = vec(collect(Iterators.product(fill([1,0],N)...)));
dw_precalc = map(spin_basis) do spin
    return domainWallL(spin, L)
end
is8 = [i for (i,dw) in enumerate(dw_precalc) if dw == 8];
spin_basis  = [spin_basis[i] for i in is8]

424-element Vector{NTuple{16, Int64}}:
 (0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
 (0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
 (1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
 (0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
 (0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
 (1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
 (0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
 (1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
 (0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
 (1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
 ⋮
 (1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
 (0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
 (1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
 (0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
 (1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
 (1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
 (0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
 (1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
 (1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [56]:
function translate_x(spin_config)
    spinMatr = Transpose(toSpinMatr(spin_config))
    res = map(vcat(2:size(spinMatr)[2],1)) do i
        return spinMatr[i,:]
    end
    return Tuple(vec(Transpose(permutedims(hcat(res...)))))
end

function translate_y(spin_config)
    spinMatr = toSpinMatr(spin_config)
    res = map(vcat(2:size(spinMatr)[2],1)) do i
        return spinMatr[i,:]
    end
    return Tuple(vec(permutedims(hcat(res...))))
end

function mirror_x(spin_config)
    spinMatr = Transpose(toSpinMatr(spin_config))
    s = size(spinMatr)[2]
    res = map(1:s) do i
        return spinMatr[s+1-i,:]
    end
    return Tuple(vec(Transpose(permutedims(hcat(res...)))))
end

function mirror_y(spin_config)
    spinMatr = toSpinMatr(spin_config)
    s = size(spinMatr)[2]
    res = map(1:s) do i
        return spinMatr[s+1-i,:]
    end
    return Tuple(vec(permutedims(hcat(res...))))
end

function parity(spin_config)
    return Tuple([(s+1)%2 for s in spin_config])
end

function transpose(spin_config)
    return Tuple(vec(Transpose(toSpinMatr(spin_config))))
end

function rotate(spin_config)
    return transpose(mirror_x(spin_config))
end

rotate (generic function with 1 method)

In [ ]:
function generate_states(states, L)
    funcs = [translate_x, translate_y, mirror_x, mirror_y, parity, transpose, rotate]
    map(states) do s
end
let 
    init_states = [
        (0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1),
        (0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1),
        (0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1),
        (0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1),
        (0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1),
        (0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1),
        (0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1),
    ]
end

# Try via perturbation theory

In [4]:
L= (4,4)
N = prod(L);
J = -1;
g = -0.1
h = -0.

next_neighbours = nearest_neighbours(L, collect(1:prod(L)))

spin_basis = vec(collect(Iterators.product(fill([1,0],N)...)));
dw_precalc = map(spin_basis) do spin
    return domainWallL(spin, L)
end

### sort basis according to domain wall length ###
sorted_spin_basis = sort(collect(zip(dw_precalc, spin_basis)), by = x->x[1])
dw_precalc  = [d[1] for d in sorted_spin_basis]
spin_basis  = [d[2] for d in sorted_spin_basis];
ts = [0.0,0.1]
step = 1.1
tmax = 1e10

### logarithmic timesteps ###
while true
    push!(ts, ts[end]*step)
    ts[end] > tmax && break
end

spin_basis_table = Dict(
    map(enumerate(zip(spin_basis,dw_precalc))) do (i, (spin, dw))
        return (spin, (i,dw))
    end
);

domainWall_table = Dict(
    map(enumerate(dw_precalc)) do (i,  dw)
        return (i, dw)
    end
);

### initial state ###
init_spin = vcat(fill(1,Int(N/2)),fill(0,Int(N/2)))
init_idx = first(spin_basis_table[Tuple(init_spin)])

init_states_indices = [145,205,251,285,336,370,416,476]


# psi=zeros(length(spin_basis))
# psi[init_idx]=1
# psi = sparse(psi)

# ### build Hamiltonians ###
H0  = build_H0(spin_basis, next_neighbours, spin_basis_table, (L,J,g,h));
H1, R1 = build_H1_R1(spin_basis, spin_basis_table, (L,J,g,h));
V1 = sparse(H1+R1);
H = H0+V1

# println("Starting caluclation of eigenvectors/-values")
# @time "eigen: " vals, vecs = eigen(Matrix(H))
# dw = map(1:size(vecs)[2]) do i
#     absVals = abs2.(vecs[:,i])
#     return sum(absVals .* dw_precalc)
# end

# dfSpec = DataFrame(en = real.(vals), dw = dw, occ = real.(vecs[init_idx,:]))
# CSV.write("./spec_Eff_L=($(L[1])_$(L[2]))_J=$(J)_g=$(g)_h=$(h).csv", dfSpec)

65536×65536 SparseMatrixCSC{Float64, Int64} with 1093572 stored entries:
⎡⣿⣿⣿⣿⣿⣗⣒⣒⡲⢶⡴⣤⣤⣤⣀⣀⢀⡀⠀⠀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⣿⣿⣿⣿⣧⡌⢛⢿⢿⣶⣙⣲⡠⠀⠉⠛⠒⠿⡧⣤⣀⣘⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢿⢿⡉⠿⢿⣷⣈⡈⡓⢮⠻⣽⣿⣤⣀⣀⢀⡀⠂⢠⡀⠙⢯⡛⢶⣦⣄⣀⡀⠀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢸⢸⣿⣔⡂⠸⣿⣿⣱⣌⠳⣄⠀⠈⢙⣿⣷⣙⡦⣄⠙⠲⣄⡀⠀⠀⠻⣧⣙⠂⠀⠦⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢸⣎⢻⣷⡹⣌⡑⢾⢿⣷⣀⢈⠳⡄⠀⠲⣜⠿⣿⣷⣀⣀⠀⣉⠐⠂⠀⠈⢹⣷⣄⣀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠐⣯⢳⣸⣟⣦⠙⢦⡀⢘⣿⣿⣓⡦⠘⠂⢤⣀⠀⠀⣻⢿⣿⣎⣓⡀⠀⠀⠀⠀⠙⠿⣦⡀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⣿⠀⠊⠛⣿⡀⠀⠙⠦⠹⡼⣿⣿⣀⣀⠀⡈⠓⢦⠈⠙⢦⡻⣿⣶⣄⣀⡈⠣⡄⠰⣌⣿⣆⡀⡀⠀⠀⠀⎥
⎢⠀⢸⣧⠀⠀⢸⣷⣴⢠⡀⠲⠀⠀⢸⡿⣯⣷⠙⢦⡀⠙⢦⡀⠀⠀⠀⠻⣷⠹⣄⠙⢦⠀⠀⢻⠳⠩⡄⠀⠀⎥
⎢⠀⠰⣼⡄⠀⠰⣝⢻⣶⡝⠀⢳⡀⠠⣝⠛⢻⣶⣦⣙⠀⠀⠙⢦⡀⠀⠙⠛⣶⣾⠂⠈⠱⠀⠘⣧⠀⠀⠀⠀⎥
⎢⠀⠀⠉⣯⠈⣀⠈⢯⢿⣿⠀⠀⠹⣄⠈⠳⣌⢻⣿⣿⠀⠀⠀⠀⠙⢦⠐⢦⢯⣿⡆⠀⠀⠘⠀⢜⡇⠀⠀⠀⎥
⎢⠀⠈⣀⢸⣄⠈⢳⡀⠀⢸⣿⣞⣆⠀⠳⣄⠀⠀⠀⠀⣿⣿⣧⡙⢦⡀⢠⠀⠀⠀⢿⣟⢾⠦⠀⠀⢻⡃⠀⠀⎥
⎢⠀⠀⠀⠈⣯⠳⠀⠹⡄⢠⡻⢿⣬⡳⠀⠈⠳⣄⠀⠀⣍⠻⠿⣧⣤⣝⠂⠰⣀⠀⢼⠿⢷⣤⡀⠀⠘⢿⠀⠀⎥
⎢⠀⠀⠀⠀⠸⣷⠀⠀⠰⠀⠙⠸⢻⣿⠀⠀⠀⠈⠳⣄⠈⠳⣄⢿⣻⣾⡇⠀⠈⢧⡈⢳⡸⣿⡆⠘⡐⣞⡇⠀⎥
⎢⠀⠀⠀⠀⠀⢹⠿⣦⡀⠀⠀⠀⠀⢹⢿⣦⣷⠀⠰⣄⠀⠒⢈⡀⠉⠉⣿⣿⣙⣦⡙⠢⣍⠉⣿⣿⡽⣌⣿⠆⎥
⎢⠀⠀⠀⠀⠀⠈⠳⠘⢷⣶⠀⠀⠦⡈⠓⢦⣸⣿⣯⣷⠀⠀⠀⠘⠦⣄⠳⣼⣿⣿⡆⠀⠈⢣⣠⣿⡅⠈⢼⠀⎥
⎢⠀⠀⠀⠀⠀⠘⠠⡄⠀⢹⣷⡄⢀⡉⠳⣄⡈⠀⠈⠉⣿⢷⣶⡗⢦⣈⠳⡈⠈⠉⣿⣿⣾⢦⡤⠉⢿⡿⠼⡇⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⣽⠀⠀⠑⠂⣀⠀⠺⡗⠙⣷⣶⣮⡇⠙⠦⣀⠺⣟⢻⣶⡏⢧⣺⣻⣦⡇⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠹⢿⡒⠶⣤⣀⢄⠀⠀⠀⠈⣈⠉⣿⣿⣤⣾⡄⠋⠯⣍⣿⣿⡟⢻⣿⣿⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠃⠦⠀⠀⠉⠉⠿⠲⣶⣄⣰⢬⡓⢯⡁⠉⣿⡷⣾⣺⣿⣉⣿⣿⣿⣿⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠻⠟⠒⠓⠶⠧⠬⠿⣿⣿⣿⣿⣿⣿⎦

In [23]:
function first_order_states(nState, basis_states, H0, V)
    inds, vals = findnz(V[nState,:])
    vals = map(zip(inds, vals)) do (i,v)
        return v/(H0[nState,nState]-H0[i,i])
    end
    return SparseVector(length(basis_states), inds, vals)
end

first_order_states(145, spin_basis, H0, V1)

65536-element SparseVector{Float64, Int64} with 16 stored entries:
  [566 ]  =  0.025
  [589 ]  =  0.025
  [603 ]  =  0.025
  [610 ]  =  0.025
  [616 ]  =  0.025
  [619 ]  =  0.025
  [621 ]  =  0.025
          ⋮
  [648 ]  =  0.025
  [674 ]  =  0.025
  [717 ]  =  0.025
  [770 ]  =  0.025
  [880 ]  =  0.025
  [962 ]  =  0.025
  [1142]  =  0.025
  [1412]  =  0.025

In [24]:
function first_order_energy(nState, basis_states, H0, V)
    return 0
end

first_order_energy(145, spin_basis, H0, V1)

0